# Lesson 02 - Convolutional Neural Networks

We will build a VGG-16 model using Keras

## Build the Model

In [1]:
import numpy as np
import os, json

In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
# We will use TensorFlow backend, thus we will explicitly set the Theano image ordering
K.set_image_dim_ordering('th')

In [4]:
# Parameters and helper functions
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.
        Args: 
            x: Image array (height x width x channels)
        Returns:
        Image array (height x width x transposed_channels)
    """
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def conv_block(n_layers, n_filters):
    """Adds a convolutional block."""
    
    for i in range(n_layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(n_filters, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

def fc_block():
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [5]:
# Build model
model = Sequential()

model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
conv_block(2, 64)
conv_block(2, 128)
conv_block(3, 256)
conv_block(3, 512)
conv_block(3, 512)
model.add(Flatten())
fc_block()
fc_block()
model.add(Dense(1000, activation='softmax'))

In [6]:
# Load weights
file_path = 'http://files.fast.ai/models/'
file_name = 'vgg16.h5'
model.load_weights(get_file(file_name, file_path+file_name, cache_subdir='models'))

In [7]:
# load imagenet class index
file_path = 'http://files.fast.ai/models/'
file_name = 'imagenet_class_index.json'
json_path = get_file(file_name, file_path+file_name, cache_subdir='models')
# load json
with open(json_path) as f:
    class_dict = json.load(f)
imagenet_classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

In [8]:
print(len(imagenet_classes))
print(imagenet_classes[:5])

1000
['tench', 'goldfish', 'great_white_shark', 'tiger_shark', 'hammerhead']


In [9]:
class_names = ['cats', 'dogs']

In [10]:
# Remove the last layer and add a layer for cat and dog
# We can remove the last added layer in a Sequential model by calling .pop()
model.pop() # removed the softmax layer
for layer in model.layers: layer.trainable=False
# add last layer
model.add(Dense(2, activation='softmax'))

In [11]:
# Compile the model
model.compile(
    optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

## Train

In [12]:
batch_size = 32

# Generators
gen = image.ImageDataGenerator()

train_batches = gen.flow_from_directory(
    'data/train-fast-ai',
    target_size=(224,224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size)
valid_batches = gen.flow_from_directory(
    'data/valid-fast-ai',
    target_size=(224,224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
print(train_batches.samples)
print(valid_batches.samples)

23000
2000


In [14]:
%mkdir -p saved/epochs

save_name = 'weights-{epoch:03d}-val_loss-{val_loss:.5f}.hdf5'

checkpoint = ModelCheckpoint(
    'saved/epochs/'+save_name, monitor='val_loss',
    verbose=1,
    save_best_only=False,
    save_weights_only=True,
    mode='min')
callbacks_list = [checkpoint]

# Train
model.fit_generator(
    train_batches, steps_per_epoch=train_batches.samples//batch_size,
    epochs=5, validation_data=valid_batches,
    validation_steps=valid_batches.samples//batch_size,
    callbacks=callbacks_list)

Epoch 1/5
718/718 [==============================] - 894s - loss: 0.4680 - acc: 0.8678 - val_loss: 0.2441 - val_acc: 0.9178
Epoch 2/5
718/718 [==============================] - 891s - loss: 0.4343 - acc: 0.8871 - val_loss: 0.2227 - val_acc: 0.9294
Epoch 3/5
718/718 [==============================] - 876s - loss: 0.4374 - acc: 0.8880 - val_loss: 0.2646 - val_acc: 0.9243
Epoch 4/5
718/718 [==============================] - 876s - loss: 0.4342 - acc: 0.8911 - val_loss: 0.2358 - val_acc: 0.9339
Epoch 5/5
718/718 [==============================] - 876s - loss: 0.4430 - acc: 0.8906 - val_loss: 0.2643 - val_acc: 0.9212


In [15]:
# save weights
%mkdir -p saved
model.save_weights('saved/saved_weights_for_full.h5')

## Predict and submit

In [38]:
batch_size = 25
gen = image.ImageDataGenerator()

test_batches = gen.flow_from_directory(
    './data/test',
    target_size=(224,224),
    class_mode=None,
    shuffle=False,
    batch_size=batch_size)

Found 12500 images belonging to 1 classes.


In [39]:
preds = model.predict_generator(test_batches, test_batches.samples//batch_size)

In [40]:
from importlib import reload
import utils
reload(utils)
from utils import submit2redux

In [41]:
submit2redux(test_batches, preds)

File saved as ./data/subm_full.csv.
